In [16]:
import os
import keras
import mlflow
import dagshub
import mlflow.keras
from pathlib import Path
import tensorflow as tf
from dotenv import load_dotenv
from dataclasses import dataclass
from urllib.parse import urlparse
from cnnClassifier.constants import *
from cnnClassifier.utils.helper import read_yaml, create_directories, save_json

In [2]:
%pwd

'c:\\Users\\jaksh\\Desktop\\Programs\\Projects\\knee-arthritis-detection\\experiments'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\jaksh\\Desktop\\Programs\\Projects\\knee-arthritis-detection'

In [4]:
load_dotenv()
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URL")
MLFLOW_TRACKING_USERNAME = os.getenv("MLFLOW_TRACKING_USERNAME")
MLFLOW_TRACKING_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD")

In [6]:
keras.models.load_model("experiments/artifacts/training/model.h5")

<Functional name=functional_1, built=True>

In [7]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    params_epochs: int


In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            path_of_model="experiments/artifacts/training/model.h5",
            training_data="experiments/artifacts/data_ingestion/Training",
            all_params=self.params,
            mlflow_uri=MLFLOW_TRACKING_URI,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            params_epochs=self.params.EPOCHS
        )
        return evaluation_config

In [20]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path_of_model: Path):
        return keras.models.load_model(path_of_model)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator, verbose=1)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("metrics.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [21]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 493 images belonging to 5 classes.


c:\Users\jaksh\anaconda3\envs\knee\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.0141 - loss: 2.5251


2024/07/16 14:05:41 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


PermissionError: [Errno 13] Unable to synchronously create file (unable to open file: name = 'C:\Users\jaksh\AppData\Local\Temp\tmp4ujixrc0\model\data\tmpv0poe3xi', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 302)